# [Mass Downloader for FDSN Compliant Web Services](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html)

It requires the following three separate things:

1. [Data Selection](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html#step-1-data-selection): The data to be downloaded can be defined by enforcing geographical or temporal constraints and a couple of other options.
2. [Storage Options](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html#step-2-storage-options): Choosing where the final MiniSEED and StationXML files should be stored.
3. [Start the Download](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html#step-3-start-the-download): Choose from which provider(s) to download and then launch the downloading process.

## [M5.9 Earthquake in Myanmar](https://earthquake.usgs.gov/earthquakes/eventpage/us70008xhp/executive)


- `2020-04-16 11:45:23` | `22.800N` | `94.029E` | `10.0 km ` | `5.9 mww`



In [1]:
import os
from obspy import UTCDateTime
from obspy.clients.fdsn.mass_downloader import RectangularDomain, Restrictions, MassDownloader

## Step 1: Data Selection

Data set selection serves the purpose to limit the data to be downloaded to data useful for the purpose at hand.

It is handled by two objects:
- subclasses of [Domain object](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.domain.html)
- [Restrictions class](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.restrictions.Restrictions.html)

## Domain

- [RectangularDomain](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.domain.RectangularDomain.html): A rectangular domain defined by latitude and longitude bounds
- [CircularDomain](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.domain.CircularDomain.html): A circular domain defined by a center point and minimum and maximum radius from that point in degrees
- [GlobalDomain](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.domain.GlobalDomain.html): Domain spanning the whole globe
- [Domain](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.domain.Domain.html): Abstract base class defining a domain - subclass it to define a new domain

In [2]:
# Rectangular Domain
domain = RectangularDomain(minlatitude=16, maxlatitude=27,
                           minlongitude=90, maxlongitude=100)

## Restrictions

Non-domain restrictions for a query

In [3]:
# event origin time
event_otime = "2020-04-16T11:45:23.000Z"
origin_time = UTCDateTime(event_otime)
event_fname = "".join(event_otime.split("T")[0].split("-")) + "".join("".join("".join(event_otime.split("T")[1].split("Z")).split(".")).split(":"))

print("orign time for output file: {}".format(event_fname))

orign time for output file: 20200416114523000


In [4]:
# Restrictions
restrictions = Restrictions(
    # time range
    starttime=origin_time - 2 * 60,
    endtime=origin_time + 10* 60,
    
    # station start and end time (default values used)
    station_starttime=None,
    station_endtime=None,
    
    # The length of one chunk in seconds (default value used)
    chunklength_in_sec=None,
    
    # network and station (default values used)
    network=None,
    station=None,
    location=None,
    channel=None,
    exclude_networks=(),
    exclude_stations=(),
    limit_stations_to_inventory=None,
    
    # If True, miniSEED files with gaps and/or overlaps will be rejected (default is True)
    reject_channels_with_gaps=False,
    
    # The minimum length of the data as a fraction of the requested time frame (0.0~1.0, default is 0.9)
    minimum_length=0.0,
    
    # Each MiniSEED file also has an associated StationXML file,
    # otherwise the MiniSEED files will be deleted afterwards (default is True)
    sanitize=False,
    
    # The minimum inter-station distance (default is 1000)
    minimum_interstation_distance_in_m=0,
    
    # Priority list for the channels.
    # Will not be used if the channel argument is used.
    channel_priorities=('BH[ZNE12]', 'BL[ZNE12]',
                        'HH[ZNE12]', 'HL[ZNE12]',
                        'SH[ZNE12]', 'SL[ZNE12]',
                        'EH[ZNE12]', 'EL[ZNE12]',
                        'SP[ZNE12]', 'EP[ZNE12]', 
                        'DP[ZNE12]'),

    # Priority list for the locations.
    # Will not be used if the location argument is used. (defaults are used)
    location_priorities=('', '00', '10', '01', '20', '02', '30', '03', '40', '04',
                         '50', '05', '60', '06', '70', '07', '80', '08', '90', '09'))


## Step 2: Storage Options

After determining what to download, the helpers must know where to store the requested data.

- [Storing MiniSEED waveforms](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html#storing-miniseed-waveforms)
- [Storing StationXML files](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html#storing-stationxml-files)

## Storing options of MiniSEED waveforms

In [5]:
# Option 1: Folder Name
# This will cause all MiniSEED files to be stored as some_folder/NETWORK.STATION.LOCATION.CHANNEL__STARTTIME__ENDTIME.mseed.
# An example of this is waveforms-op1/BW.FURT..BHZ__20141027T163723Z__20141027T163733Z.mseed.
#mseed_storage = "waveforms-op1"


# Option 2: String Template
# An example of this is some_folder/BW/FURT/BHZ..20141027T163723Z.20141027T163733Z.mseed.
#mseed_storage = ("wavefroms-op2/{network}/{station}/{channel}.{location}.{starttime}.{endtime}.mseed")


# Option 3: Custom Function
# we need to write our own is_in_db function
def get_mseed_storage(network, station, location, channel, starttime,
                      endtime):
    # Returning True means that neither the data nor the StationXML file will be downloaded.
    #if is_in_db(network, station, location, channel, starttime, endtime):
    #    return True
    # If a string is returned the file will be saved in that location.
    return os.path.join("waveforms-op3/{}".format(event_fname), "{}.{}.{}.{}.mseed".format(network, station,location, channel))

mseed_storage = get_mseed_storage

## Storing options of StationXML files

In [6]:
# Option 1: Folder Name
# This example will save the files to "stations-op1/NETWORK.STATION.xml", e.g. to "stations/BW.FURT.xml".
stationxml_storage = "stations-op1"


# Option 2: String Template
# This example will write to "some_folder/NETWORK/STATION.xml", in this case for example to "some_folder/BW/FURT.xml".
#stationxml_storage = "station-op2/{network}/{station}.xml"


# Option 3: Custom Function
# we need to write our own is_in_db function
#def get_stationxml_storage(network, station, channels, starttime, endtime):
#    available_channels = []
#    missing_channels = []
#    for location, channel in channels:
#        if is_in_db(network, station, location, channel, starttime, endtime):
#            available_channels.append((location, channel))
#        else:
#            missing_channels.append((location, channel))
#    filename = os.path.join("stations-op3", "{}.{}.xml".format(network, station))
#    return {
#        "available_channels": available_channels,
#        "missing_channels": missing_channels,
#        "filename": filename}
#
#stationxml_storage = get_stationxml_storage

## Step 3: Start the Download

The final step is to actually start the download. Pass the previously created domain, restrictions, and path settings and off you go.

In [7]:
# List of FDSN client names or service URLS
# Will use all FDSN implementations known to ObsPy except RASPISHAKE if set to None
# The order in the list also determines their priority
# To include RASPISHAKE, you must set this parameter to obspy.clients.fdsn.header.URL_MAPPINGS explicitly
#mdl = MassDownloader(providers=["IRIS", "GFZ", "SCEDC"])
mdl = MassDownloader()

# domain, restrictions, mseed_storage, stationxml_storage,
# download_chunk_size_in_mb=20, threads_per_client=3, print_report=True (default)
tt = mdl.download(domain, restrictions, threads_per_client=3, 
                  mseed_storage=mseed_storage, stationxml_storage=stationxml_storage)

[2020-12-13 23:58:37,509] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for BGR, EMSC, ETH, GEONET, GFZ, ICGC, INGV, IPGP, ISC, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, SCEDC, TEXNET, UIB-NORSAR, USGS, USP, ORFEUS, IRIS.
[2020-12-13 23:58:38,175] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'ISC' as it does not have 'dataselect' and/or 'station' services.
[2020-12-13 23:58:38,905] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'EMSC' as it does not have 'dataselect' and/or 'station' services.
[2020-12-13 23:58:38,981] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'USGS' as it does not have 'dataselect' and/or 'station' services.
[2020-12-13 23:58:40,384] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 20 client(s): BGR, ETH, GEONET, GFZ, ICGC, INGV, IPGP, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, SCEDC, TEXNET, UIB-NORSAR, USP, ORFEUS, IRIS.
[2020-12-13 23:58:40,387] - obspy.clients.f

[2020-12-13 23:58:47,633] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NOA' - No data available for request.
[2020-12-13 23:58:47,634] - obspy.clients.fdsn.mass_downloader - INFO: Client 'NOA' - No data available.
[2020-12-13 23:58:47,635] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 1
[2020-12-13 23:58:47,636] - obspy.clients.fdsn.mass_downloader - INFO: Client 'RESIF' - Requesting reliable availability.
[2020-12-13 23:58:48,422] - obspy.clients.fdsn.mass_downloader - INFO: Client 'RESIF' - No data available for request.
[2020-12-13 23:58:48,423] - obspy.clients.fdsn.mass_downloader - INFO: Client 'RESIF' - No data available.
[2020-12-13 23:58:48,424] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 1
[2020-12-13 23:58:48,425] - obspy.clients.fdsn.mass_downloader - INFO: Client 'SCEDC' - Requesting unreliable availability.
[2020-12-13 23:58:49,634] - obspy.clients.fdsn.mass_downloader - INFO: Client 'S

[2020-12-13 23:59:54,776] - obspy.clients.fdsn.mass_downloader - INFO: Client 'GEONET' - Acquired 0 StationXML files [0.0 MB].
[2020-12-13 23:59:54,777] - obspy.clients.fdsn.mass_downloader - INFO: Client 'GFZ' - Acquired 3 MiniSEED files [0.1 MB].
[2020-12-13 23:59:54,778] - obspy.clients.fdsn.mass_downloader - INFO: Client 'GFZ' - Acquired 1 StationXML files [0.3 MB].
[2020-12-13 23:59:54,779] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ICGC' - Acquired 0 MiniSEED files [0.0 MB].
[2020-12-13 23:59:54,779] - obspy.clients.fdsn.mass_downloader - INFO: Client 'ICGC' - Acquired 0 StationXML files [0.0 MB].
[2020-12-13 23:59:54,780] - obspy.clients.fdsn.mass_downloader - INFO: Client 'INGV' - Acquired 0 MiniSEED files [0.0 MB].
[2020-12-13 23:59:54,781] - obspy.clients.fdsn.mass_downloader - INFO: Client 'INGV' - Acquired 0 StationXML files [0.0 MB].
[2020-12-13 23:59:54,782] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IPGP' - Acquired 0 MiniSEED files [0.0 MB].
[2020-12